Data source - https://www.kaggle.com/dalpozz/creditcardfraud

Description -
The datasets contains transactions made by credit cards in September 2013 by european cardholders. This dataset presents transactions that occurred in two days, where we have 492 frauds out of 284,807 transactions. The dataset is highly unbalanced, the positive class (frauds) account for 0.172% of all transactions.

It contains only numerical input variables which are the result of a PCA transformation. Unfortunately, due to confidentiality issues, the original features and more background information about the data is not included. Features V1, V2, ... V28 are the principal components obtained with PCA, the only features which have not been transformed with PCA are 'Time' and 'Amount'. Feature 'Time' contains the seconds elapsed between each transaction and the first transaction in the dataset. The feature 'Amount' is the transaction Amount, this feature can be used for example-dependant cost-senstive learning. Feature 'Class' is the response variable and it takes value 1 in case of fraud and 0 otherwise.

The dataset was collected and analysed during a research collaboration of Worldline and the Machine Learning Group (http://mlg.ulb.ac.be) of ULB (Université Libre de Bruxelles) on big data mining and fraud detection. More details on current and past projects on related topics are available on http://mlg.ulb.ac.be/BruFence and http://mlg.ulb.ac.be/ARTML

Citation: Andrea Dal Pozzolo, Olivier Caelen, Reid A. Johnson and Gianluca Bontempi. Calibrating Probability with Undersampling for Unbalanced Classification. In Symposium on Computational Intelligence and Data Mining (CIDM), IEEE, 2015

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

df = pd.read_csv("creditcardfraud/creditcard.csv")

In [2]:
df_class = df["Class"]
df = df.drop("Class", axis=1)
df = df.drop("Time", axis=1)

scaler = MinMaxScaler(feature_range=(0, 1))
amount = df['Amount']
amount = amount.reshape((len(amount), 1))
scaled_amount = scaler.fit_transform(amount)
df['Amount'] = scaled_amount

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: reshape is deprecated and will raise in a subsequent release. Please use .values.reshape(...) instead
  import sys


In [3]:
data_train, data_test, class_train, class_test = train_test_split(df, df_class, test_size=0.4, random_state=0)

In [4]:
print("Training Dataset")
print("Genuine transactions - ", len(class_train[class_train == 0]))
print("Fradulent transactions - ", len(class_train[class_train == 1]))

Training Dataset
Genuine transactions -  170591
Fradulent transactions -  293


In [5]:
print("Test Dataset")
print("Genuine transactions - ", len(class_test[class_test == 0]))
print("Fradulent transactions - ", len(class_test[class_test == 1]))

Test Dataset
Genuine transactions -  113724
Fradulent transactions -  199


## Random Forest

Make sure imblearn is installed (pip install imblearn) as it is not included by default in Anaconda.

In [6]:
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix

In [7]:
oversampler = SMOTE(random_state=0, ratio=0.5, kind='svm', k_neighbors=8)
os_train, os_class = oversampler.fit_sample(data_train, class_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [8]:
print("SMOTE Adjusted Training Dataset")
print("Genuine transactions - ", len(os_class[os_class == 0]))
print("Fradulent transactions - ", len(os_class[os_class == 1]))

SMOTE Adjusted Training Dataset
Genuine transactions -  170591
Fradulent transactions -  85295


In [9]:
rf_classifier = RandomForestClassifier(n_estimators=50, random_state=0, max_features='sqrt')
rf_classifier.fit(os_train, os_class)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='sqrt', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=50, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [10]:
rf_predictions = rf_classifier.predict(data_test)
rf_cm = confusion_matrix(class_test, rf_predictions)

In [11]:
columns = ["Predicted As Genuine Transactions", "Predicted As Fradulent Transactions"]
index = ["Actual Genuine Transactions", "Actual Fraudulent Transactions"]
df_rf = pd.DataFrame(rf_cm, index=index, columns=columns)
df_rf

,Predicted As Genuine Transactions,Predicted As Fradulent Transactions
Actual Genuine Transactions,113700,24
Actual Fraudulent Transactions,33,166


In [12]:
test_predictions = pd.DataFrame()
test_predictions['RFpredictions'] = rf_predictions

In [13]:
train_data = data_train.as_matrix()
rf_predictions = rf_classifier.predict(train_data)
train_predictions = pd.DataFrame()
train_predictions['RFprediction'] = rf_predictions

### XGBoost

In [14]:
oversampler = SMOTE(random_state=0, ratio=0.5)
os_train, os_class = oversampler.fit_sample(data_train, class_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:75: DeprecationWarning: Function _ratio_float is deprecated; Use a float for 'ratio' is deprecated from version 0.2. The support will be removed in 0.4. Use a dict, str, or a callable instead.
  warnings.warn(msg, category=DeprecationWarning)


In [15]:
print("SMOTE Adjusted Training Dataset")
print("Genuine transactions - ", len(os_class[os_class == 0]))
print("Fradulent transactions - ", len(os_class[os_class == 1]))

SMOTE Adjusted Training Dataset
Genuine transactions -  170591
Fradulent transactions -  85295


In [16]:
import xgboost as xgb
xgb_classifier = xgb.XGBClassifier(n_estimators=100, random_state=0, max_depth=15, min_child_weight=2.5, gamma=1.5,
                                  objective='binary:logitraw', nthread=16, learning_rate=0.1, subsample=0.5,
                                  booster='gbtree', colsample_bytree=0.5, colsample_bylevel=0.5, reg_lambda=0,
                                   reg_alpha=0, max_delta_step=0, seed=0)

xgb_classifier.fit(os_train, os_class)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.5,
       colsample_bytree=0.5, gamma=1.5, learning_rate=0.1,
       max_delta_step=0, max_depth=15, min_child_weight=2.5, missing=None,
       n_estimators=100, n_jobs=1, nthread=16, objective='binary:logitraw',
       random_state=0, reg_alpha=0, reg_lambda=0, scale_pos_weight=1,
       seed=0, silent=True, subsample=0.5)

In [17]:
test_array = data_test.as_matrix()
xgb_predictions = xgb_classifier.predict(test_array)
xgb_cm = confusion_matrix(class_test, xgb_predictions)

In [18]:
df_xgb = pd.DataFrame(xgb_cm, index=index, columns=columns)
df_xgb

,Predicted As Genuine Transactions,Predicted As Fradulent Transactions
Actual Genuine Transactions,113698,26
Actual Fraudulent Transactions,32,167


In [19]:
test_predictions['XGBprediction'] = xgb_predictions

In [20]:
train_data = data_train.as_matrix()
xgb_predictions =  xgb_classifier.predict(train_data)
train_predictions['XGBprediction'] = xgb_predictions

## Autoencoder

In [21]:
from keras.models import Sequential
from keras.layers import Dense, Activation

np.random.seed(0)
epoch = 50

AE1 = Sequential([
        Dense(24, input_shape=(28,)),
        Activation('relu'),
        Dense(28),
        Activation('relu'),
    ])

AE1.compile(optimizer='sgd', loss='mse', metrics=['accuracy'])

Using TensorFlow backend.


In [22]:
data_train = data_train.drop("Amount", axis=1)
data_test = data_test.drop("Amount", axis=1)

os_train = data_train.as_matrix()
os_class = class_train.as_matrix()

In [23]:
from keras.callbacks import ModelCheckpoint, TensorBoard
AE1checkpointer = ModelCheckpoint(filepath="AE1", verbose=0, save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
AE1.fit(os_train, os_train, epochs=epoch, batch_size=16, shuffle=True, validation_data=(os_train, os_train),
                      verbose=1, callbacks=[AE1checkpointer, tensorboard])

Train on 170884 samples, validate on 170884 samples
Epoch 1/50
170884/170884 [==============================] - 11s - loss: 0.8442 - acc: 0.5667 - val_loss: 0.7536 - val_acc: 0.6955
Epoch 2/50
170884/170884 [==============================] - 11s - loss: 0.7371 - acc: 0.7291 - val_loss: 0.7242 - val_acc: 0.7547
Epoch 3/50
170884/170884 [==============================] - 11s - loss: 0.7161 - acc: 0.7685 - val_loss: 0.7092 - val_acc: 0.7802
Epoch 4/50
170884/170884 [==============================] - 11s - loss: 0.7027 - acc: 0.7905 - val_loss: 0.6969 - val_acc: 0.8015
Epoch 5/50
170884/170884 [==============================] - 11s - loss: 0.6923 - acc: 0.8115 - val_loss: 0.6873 - val_acc: 0.8200
Epoch 6/50
170884/170884 [==============================] - 11s - loss: 0.6827 - acc: 0.8280 - val_loss: 0.6771 - val_acc: 0.8372cc: 0 - ETA: 4s - loss: 0.6956 - ac - ETA: 3s - loss:  - ETA: 2 - ETA: 1s - los
Epoch 7/50
170884/170884 [==============================] - 11s - loss: 0.6690 - acc: 0.8

In [24]:
from keras import backend as K
AE1_get_activations = K.function([AE1.layers[0].input], [AE1.layers[1].output])
AE1_features = AE1_get_activations([os_train])[0]

In [25]:
Classifier = Sequential([
        Dense(1, input_shape=(24,)),
        Activation('sigmoid'),
    ])

Classifier.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

In [26]:
classifier_checkpointer = ModelCheckpoint(filepath="Classifier", verbose=0, save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
Classifier.fit(AE1_features, os_class, epochs=10, batch_size=16, shuffle=True, validation_data=(AE1_features, os_class),
                      verbose=1, callbacks=[classifier_checkpointer, tensorboard])

Train on 170884 samples, validate on 170884 samples
Epoch 1/10
170884/170884 [==============================] - 11s - loss: 0.0149 - acc: 0.9986 - val_loss: 0.0089 - val_acc: 0.9991
Epoch 2/10
170884/170884 [==============================] - 11s - loss: 0.0081 - acc: 0.9992 - val_loss: 0.0073 - val_acc: 0.9992
Epoch 3/10
170884/170884 [==============================] - 11s - loss: 0.0071 - acc: 0.9992 - val_loss: 0.0069 - val_acc: 0.9993 1s - loss: 0.0076 - acc - ETA: 0s - loss: 0.0073
Epoch 4/10
170884/170884 [==============================] - 11s - loss: 0.0068 - acc: 0.9993 - val_loss: 0.0066 - val_acc: 0.9993A: 4s - loss: 0.0065 - a - ETA: 4s - loss: 0.0070 - ETA: 3s - loss: 0.0073 -  - ETA: 0s - loss: 0.0069 - ac
Epoch 5/10
170884/170884 [==============================] - 11s - loss: 0.0066 - acc: 0.9993 - val_loss: 0.0064 - val_acc: 0.9993
Epoch 6/10
170884/170884 [==============================] - 12s - loss: 0.0064 - acc: 0.9993 - val_loss: 0.0063 - val_acc: 0.9993 - ETA: 4s - 

In [27]:
DenseNetwork = Sequential()
DenseNetwork.add(AE1.layers[0])
DenseNetwork.add(AE1.layers[1])
DenseNetwork.add(Classifier)

In [28]:
from keras import optimizers
sgd = optimizers.SGD(lr=0.01, clipnorm=1.)
DenseNetwork.compile(optimizer=sgd, loss='binary_crossentropy', metrics=['accuracy'])

In [29]:
DenseNetwork.fit(os_train, os_class, epochs=5, batch_size=16, shuffle=True, validation_data=(os_train, os_class), verbose=1)

Train on 170884 samples, validate on 170884 samples
Epoch 1/5
170884/170884 [==============================] - 13s - loss: 0.0055 - acc: 0.9993 - val_loss: 0.0051 - val_acc: 0.9994
Epoch 2/5
170884/170884 [==============================] - 12s - loss: 0.0050 - acc: 0.9994 - val_loss: 0.0048 - val_acc: 0.9994
Epoch 3/5
170884/170884 [==============================] - 13s - loss: 0.0047 - acc: 0.9994 - val_loss: 0.0045 - val_acc: 0.9994
Epoch 4/5
170884/170884 [==============================] - 12s - loss: 0.0044 - acc: 0.9994 - val_loss: 0.0042 - val_acc: 0.9994
Epoch 5/5
170884/170884 [==============================] - 12s - loss: 0.0041 - acc: 0.9994 - val_loss: 0.0041 - val_acc: 0.9994loss: 0.0045 - ac - ETA: 0s - loss: 0.0043


In [30]:
data = data_test.as_matrix()
prob_threshold = 0.05

predictions_ae = DenseNetwork.predict(data)
predictions = DenseNetwork.predict(data)
predictions_ae[predictions_ae > prob_threshold] = 1
predictions_ae[predictions_ae != 1] = 0
cm_ae = confusion_matrix(class_test, predictions_ae)
columns = ["Predicted As Genuine Transactions", "Predicted As Fradulent Transactions"]
index = ["Actual Genuine Transactions", "Actual Fraudulent Transactions"]
df_ae = pd.DataFrame(cm_ae, index=index, columns=columns)
df_ae

,Predicted As Genuine Transactions,Predicted As Fradulent Transactions
Actual Genuine Transactions,113679,45
Actual Fraudulent Transactions,36,163


In [31]:
test_predictions['AEprediction'] = predictions

In [32]:
ae_predictions =  DenseNetwork.predict(os_train)
train_predictions['AEprediction'] = ae_predictions

## Ensemble

In [33]:
train_predictions_matrix = train_predictions.as_matrix()


Ensemble = Sequential([
        Dense(1, input_shape=(3,)),
        Activation('sigmoid'),
    ])

Ensemble.compile(optimizer='sgd', loss='binary_crossentropy', metrics=['accuracy'])

classifier_checkpointer = ModelCheckpoint(filepath="Classifier", verbose=0, save_best_only=True)
tensorboard = TensorBoard(log_dir='./logs', histogram_freq=0, write_graph=True, write_images=True)
Ensemble.fit(train_predictions_matrix, os_class, epochs=50, batch_size=16, shuffle=True,
             validation_data=(train_predictions_matrix, os_class), verbose=1,
             callbacks=[classifier_checkpointer, tensorboard])

Train on 170884 samples, validate on 170884 samples
Epoch 1/50
170884/170884 [==============================] - 12s - loss: 0.0520 - acc: 0.9982 - val_loss: 0.0154 - val_acc: 0.9983.9 - ETA: 6s - lo
Epoch 2/50
170884/170884 [==============================] - 11s - loss: 0.0126 - acc: 0.9983 - val_loss: 0.0109 - val_acc: 0.9983
Epoch 3/50
170884/170884 [==============================] - 12s - loss: 0.0099 - acc: 0.9983 - val_loss: 0.0091 - val_acc: 0.9983
Epoch 4/50
170884/170884 [==============================] - 11s - loss: 0.0085 - acc: 0.9983 - val_loss: 0.0080 - val_acc: 0.9983
Epoch 5/50
170884/170884 [==============================] - 12s - loss: 0.0075 - acc: 0.9983 - val_loss: 0.0071 - val_acc: 0.9983
Epoch 6/50
170884/170884 [==============================] - 11s - loss: 0.0067 - acc: 0.9983 - val_loss: 0.0063 - val_acc: 0.9983
Epoch 7/50
170884/170884 [==============================] - 12s - loss: 0.0059 - acc: 0.9983 - val_loss: 0.0056 - val_acc: 0.9983.9 - ETA: 6s - loss: 0

In [34]:
test_predictions_matrix = test_predictions.as_matrix()
prob_threshold = 0.5
predictions = Ensemble.predict(test_predictions_matrix)
predictions[predictions > prob_threshold] = 1
predictions[predictions != 1] = 0
cm = confusion_matrix(class_test, predictions)
columns = ["Predicted As Genuine Transactions", "Predicted As Fradulent Transactions"]
index = ["Actual Genuine Transactions", "Actual Fraudulent Transactions"]
df_ensemble = pd.DataFrame(cm, index=index, columns=columns)
df_ensemble

,Predicted As Genuine Transactions,Predicted As Fradulent Transactions
Actual Genuine Transactions,113707,17
Actual Fraudulent Transactions,34,165
